In [1]:
using Rays

## Creating a scene

In [2]:
# Define the scene
scene = Rays.Scene()

# Define a camera
camera = Rays.Camera()
camera.screen_res .= [500, 500]
push!(scene, camera)

# Define a cube at the origin
cube = Rays.Cube(zeros(Float32, 3), 1.0f0)
push!(scene, cube)

## Creating a parameters object

In [8]:
julia_green = Float32[0.22, 0.596, 0.149]
julia_purple = Float32[0.584, 0.345, 0.698]
julia_red = Float32[0.796, 0.235, 0.2]
julia_colors = hcat(julia_green, julia_purple, julia_red)
color = Rays.get_canvas(camera; color = true)

mutable struct Params{F} <: Rays.Parameters{F}
    θ::F
    ϕ::F
    dθ::F
    dϕ::F
    dist::F
    colors::Matrix{F}
    color::Array{F,3}
end

params = Params(Float32(π/4), Float32(3π/8), 0.0f0, 0.0f0, 10.0f0, julia_colors, color)

Params{Float32}(0.7853982f0, 1.1780972f0, 0.0f0, 0.0f0, 10.0f0, Float32[0.22 0.584 0.796; 0.596 0.345 0.235; 0.149 0.698 0.2], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; … ;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0])

## Creating a parameter update function

In [9]:
import SimpleDirectMediaLayer as SDL

function affect_parameters!(params::Params{F}, Δt::F)::Tuple{Bool,Bool} where {F}
    event_ref = Ref{SDL.SDL_Event}()
    close = false
    while Bool(SDL.SDL_PollEvent(event_ref))
        event = event_ref[]
        event_type = event.type
        if event_type == SDL.SDL_QUIT
            close = true
            break
        elseif event_type == SDL.SDL_KEYDOWN
            scan_code = event.key.keysym.scancode 
            if scan_code == SDL.SDL_SCANCODE_RIGHT
                params.dθ += 0.5*Δt
            elseif scan_code == SDL.SDL_SCANCODE_LEFT
                params.dθ -= 0.5*Δt
            end
        end
    end

    params.θ += params.dθ * Δt
    params.ϕ += params.dϕ * Δt
    return true, close
end

affect_parameters! (generic function with 1 method)

## Creating a rendering function

In [10]:
function get_render(scene::Rays.Scene{F}, params::Rays.Parameters{F})::Array{F, 3} where {F <: AbstractFloat}

    # Adjust camera
    Rays.look_at!(camera, zeros(Float32,3), params.dist, params.θ, params.ϕ)

    # Get intersection data
    intersection_data = Rays.shape_view(scene; data_variables = [:dim])

    # Get grayscale image
    canvas_grayscale = Rays.cam_is_source(intersection_data.t);

    # Apply color to the image
    color .= 0.0
    Rays.add_color!(color, julia_colors, intersection_data.dim)
    canvas_color = Rays.apply_color(canvas_grayscale, color);

    return canvas_color
end 

get_render (generic function with 1 method)

## Creating an interactor

In [11]:
interactor = Rays.Interactor(scene, params, affect_parameters!, get_render)

Rays.Interactor{Float32}(Ptr{SimpleDirectMediaLayer.LibSDL2.SDL_Window} @0x000001c1b3ad8210, Ptr{SimpleDirectMediaLayer.LibSDL2.SDL_Renderer} @0x000001c1b4029c40, Rays.Scene{Float32}(Rays.Camera{Float32}[Rays.Camera{Float32}(Float32[0.0, 0.0, 0.0], Float32[1.0, 0.0, 0.0], Float32[0.0, 0.0, 1.0], Float32[0.0, -1.0, 0.0], Float32[0.1, 0.1], Float32[0.1], [500, 500], identity)], Rays.Shape{Float32}[Rays.Cube{Float32}(Float32[0.0, 0.0, 0.0], 1.0f0)]), Params{Float32}(0.7853982f0, 1.1780972f0, 0.0f0, 0.0f0, 10.0f0, Float32[0.22 0.584 0.796; 0.596 0.345 0.235; 0.149 0.698 0.2], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; … ;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]), get_render, affect_parameters!, Rays.Dtimer{Float32}(0.0f0))

In [12]:
Rays.run!(interactor)